In [1956]:
import pandas as pd
import numpy as np
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [1957]:
#start_time = time.time()
#Phase1 Using the offline workload A as the traning data

#Step1 Load data

workload_A_all=pd.read_csv('../../offline_workload.csv') 
#print(workload_A.head())
tmp_flag = (workload_A_all != 0).any(axis=0)
workload_A = workload_A_all.loc[:, tmp_flag]
#workload_A
workload_A = workload_A.loc[:, ~(workload_A == workload_A.iloc[0]).all()]

workload_A = workload_A.loc[:, workload_A.isin([0]).mean() < .6]


#print(workload_A_all)
#print("--- %s seconds ---" % (time.time() - start_time))

In [1958]:
start_time = time.time()

#Step2 Split workload_A by creating different dataframes

workload_A_ids = workload_A["workload id"].unique()
workload_A_list = []
for num,workload_A_id in enumerate(workload_A_ids,start=1):
    tmp_workload = workload_A[workload_A['workload id'].isin([workload_A_id])]
    workload_A_list.append(tmp_workload)
    exec("workload_A_%s = tmp_workload"%num)

#workload_A_1
print("--- %s seconds ---" % (time.time() - start_time))
workload_A_list[0]

--- 0.07678341865539551 seconds ---


,workload id,k1,k2,k3,k4,k5,k6,k7,k8,s1,...,worker_2.Paging_and_Virtual_Memory.pgactivate,worker_2.Paging_and_Virtual_Memory.pgalloc_normal,worker_2.Paging_and_Virtual_Memory.pgfault,worker_2.Paging_and_Virtual_Memory.pgfree,worker_2.Paging_and_Virtual_Memory.pgpgin,worker_2.Paging_and_Virtual_Memory.pgpgout,worker_2.Processes.Blocked,worker_2.Processes.Runnable,worker_2.Processes.fork,worker_2.Processes.pswitch
0,1-7,12,2,2,4,48,200,True,0.60,10000,...,4424.279221,18262.266775,12853.448593,16797.275433,72.181818,7008.308442,0.069805,3.165043,23.681602,24992.673268
1,1-7,144,18,4,8,48,200,False,0.60,10000,...,31109.787500,108408.287500,74437.975000,105656.075000,1075.200000,45536.437500,1.150000,16.475000,106.908750,76874.642500
2,1-7,64,8,4,8,48,200,True,0.55,10000,...,17055.333333,55772.050000,49196.275000,55277.691667,1063.233333,27153.891667,0.316667,10.791667,91.712500,67034.970833
3,1-7,72,8,3,6,384,200,True,0.60,10000,...,14254.089286,43482.345238,39937.422619,45322.690476,0.000000,25124.755952,0.291667,8.708333,75.976786,52007.676786
4,1-7,16,4,2,4,48,200,False,0.60,10000,...,12994.062500,28767.520833,18894.458333,26735.541667,0.000000,18149.208333,0.562500,4.625000,42.437500,34945.314583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,1-7,144,24,3,6,96,200,True,0.60,1000,...,19171.758333,74284.575000,72534.700000,70025.929167,550.033333,44299.208333,0.254167,16.804167,104.235417,73872.259167
317,1-7,144,24,3,6,96,200,True,0.60,20000,...,19820.829167,75654.862500,70649.587500,72247.612500,571.866667,44799.662500,0.254167,15.195833,106.215417,75531.375000
318,1-7,144,36,2,4,24,200,True,0.60,10000,...,19706.866667,96354.691667,98544.100000,89583.654167,284.883333,40907.779167,0.279167,18.112500,104.513333,76792.792917
319,1-7,216,24,3,6,24,200,True,0.60,10000,...,18967.933333,69006.175000,65629.816667,66646.429167,620.266667,44582.054167,0.383333,15.795833,99.993333,72002.147917


In [1]:
start_time = time.time()

#Step3 Get metrics

tmpframe = workload_A.T.copy()
#tmpframe = tmpframe.tail(585-1-12).T
#tmpframe = tmpframe.tail(414-1-12).T
#tmpframe = tmpframe.tail(372-1-12).T
tmpframe = tmpframe.tail(314-1-12).T
metrics_name = tmpframe.columns
#print(metrics_name)
workload_A_listformetrics = []
num_workloads_A = len(workload_A_list)
for i in range(num_workloads_A):
    tmp = (workload_A_list[i])[metrics_name]
    workload_A_listformetrics.append(tmp)

print("--- %s seconds ---" % (time.time() - start_time))
(tmpframe)

NameError: name 'time' is not defined

In [1960]:
#Step4 FA for all workloads together
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize


start_time = time.time()
all_metrics_data = tmpframe.values

#all_metrics_data_Trans = all_metrics_data.T

all_metrics_data_normalized = normalize(all_metrics_data, norm='l2')
all_metrics_data_normalized_Trans = all_metrics_data_normalized.T

#all_metrics_data_Trans = all_metrics_data.T
#all_metrics_data_normalized_Trans = normalize(all_metrics_data_Trans, norm='max')

print(all_metrics_data_normalized_Trans.shape)
tmp_all_transformer = FactorAnalysis(n_components=150, random_state=0)
tmp_workload_A_transformed = tmp_all_transformer.fit_transform(all_metrics_data_Trans)
print(tmp_workload_A_transformed.shape)
print("--- %s seconds ---" % (time.time() - start_time))


(301, 18349)
(301, 150)
--- 2.3839058876037598 seconds ---


In [1961]:
#Step5 K-means for all workloads together
from sklearn.cluster import KMeans

num_M = 4

workload_A_list_array_Kmeans = KMeans(n_clusters=num_M, random_state=0).fit(tmp_workload_A_transformed)
cluster_centers = workload_A_list_array_Kmeans.cluster_centers_
tmp_centers_index = []
tmp_centers_metrics_name = []
for i in range(num_M):
    tmp_center = cluster_centers[i]
    tmp_center_diff = tmp_center - tmp_workload_A_transformed
    tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)
    tmp_center_min_idx = np.argmin(tmp_center_distance)
    tmp_centers_index.append(tmp_center_min_idx)
    
for j in range(num_M):
        tmp_mname = metrics_name[tmp_centers_index[j]]
        #print(tmp_mname)
        tmp_centers_metrics_name.append(tmp_mname)
print(tmp_centers_index)
print(tmp_centers_metrics_name)
#tmp_centers_metrics_name.append('latency')
tmp_centers_metrics_name = ['latency'] + tmp_centers_metrics_name
print(tmp_centers_metrics_name)
num_M += 1
workload_list_centers_metrics_name = []
for n in range(58):
    workload_list_centers_metrics_name.append(tmp_centers_metrics_name)


[157, 294, 119, 117]
['worker_1.Disk_Block_Size.dm-0', 'worker_2.Paging_and_Virtual_Memory.pgfree', 'executor.jvm.pools.PS-Old-Gen.max.avg', 'executor.jvm.pools.PS-Eden-Space.used.avg']
['latency', 'worker_1.Disk_Block_Size.dm-0', 'worker_2.Paging_and_Virtual_Memory.pgfree', 'executor.jvm.pools.PS-Old-Gen.max.avg', 'executor.jvm.pools.PS-Eden-Space.used.avg']


In [1962]:
#quick_view_metric = workload_A[tmp_centers_metrics_name]
#quick_view_metric

In [1963]:
'''start_time = time.time()

#Step4 FA (apply for all workloads)
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize

#transformer = FactorAnalysis(n_components=7, random_state=0)
workload_list_afterFA = []

for i in range(58):
    tmp_workload_A_listformetrics = workload_A_listformetrics[i]
    tmp_workload_A_listformetrics_array = tmp_workload_A_listformetrics.values
    #print(tmp_workload_A_listformetrics)
    tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array, norm='l2')
    tmp_workload_A_listformetrics_array_normalized_Trans = tmp_workload_A_listformetrics_array_normalized.T
    
    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T
    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T
    #tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array_Trans, norm='l2')
    tmp_transformer = FactorAnalysis(n_components=20, random_state=0)
    #print(tmp_workload_A_listformetrics_array_Trans)
    #normalize tmp_workload_A_listformetrics_array_Trans
    #tmp_workload_A_listformetrics_array_Trans_normalized = normalize(tmp_workload_A_listformetrics_array_Trans, norm='l2')
    #print('-------------------------')
    #print(tmp_workload_A_listformetrics_array_Trans_normalized)
    
    tmp_workload_A_listformetrics_array_Trans_transformed = tmp_transformer.fit_transform(tmp_workload_A_listformetrics_array_normalized_Trans)
    workload_list_afterFA.append(tmp_workload_A_listformetrics_array_Trans_transformed)

print("--- %s seconds ---" % (time.time() - start_time))
#print(workload_list_afterFA[0].shape)
'''

'start_time = time.time()\n\n#Step4 FA (apply for all workloads)\nfrom sklearn.decomposition import FactorAnalysis\nfrom sklearn.preprocessing import normalize\n\n#transformer = FactorAnalysis(n_components=7, random_state=0)\nworkload_list_afterFA = []\n\nfor i in range(58):\n    tmp_workload_A_listformetrics = workload_A_listformetrics[i]\n    tmp_workload_A_listformetrics_array = tmp_workload_A_listformetrics.values\n    #print(tmp_workload_A_listformetrics)\n    tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array, norm=\'l2\')\n    tmp_workload_A_listformetrics_array_normalized_Trans = tmp_workload_A_listformetrics_array_normalized.T\n    \n    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T\n    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T\n    #tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array_Trans, norm=\'l2\')\n    tmp_

In [1964]:
'''start_time = time.time()

#Step5 K-means (apply for all workloads)
from sklearn.cluster import KMeans

num_M = 2

workload_list_centers_metrics_name = []

for i in range(58):
    tmp_workload_A_listformetrics_array_Kmeans = KMeans(n_clusters=num_M, random_state=0).fit(workload_list_afterFA[i])
    tmp_cluster_centers = tmp_workload_A_listformetrics_array_Kmeans.cluster_centers_
    #print(tmp_cluster_centers)
    tmp_centers_index = []
    tmp_centers_metrics_name= []
    #print(workload_list_afterFA[i].shape)
    for j in range(num_M):
        tmp_center = tmp_cluster_centers[j]
        tmp_center_diff = tmp_center - workload_list_afterFA[i]
        tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)
        tmp_center_min_idx = np.argmin(tmp_center_distance)
        tmp_centers_index.append(tmp_center_min_idx)
        #print(tmp_center_min_idx)
        #print('-----')
    #tmp_centers_metrics_name = []
    tmp_n_centers = len(tmp_cluster_centers)
    #print(tmp_n_centers)
    for j in range(tmp_n_centers):
        tmp_mname = metrics_name[tmp_centers_index[j]]
        #print(tmp_mname)
        tmp_centers_metrics_name.append(tmp_mname)
    
    workload_list_centers_metrics_name.append(tmp_centers_metrics_name)

print("--- %s seconds ---" % (time.time() - start_time))
print(workload_list_centers_metrics_name[0])
'''

'start_time = time.time()\n\n#Step5 K-means (apply for all workloads)\nfrom sklearn.cluster import KMeans\n\nnum_M = 2\n\nworkload_list_centers_metrics_name = []\n\nfor i in range(58):\n    tmp_workload_A_listformetrics_array_Kmeans = KMeans(n_clusters=num_M, random_state=0).fit(workload_list_afterFA[i])\n    tmp_cluster_centers = tmp_workload_A_listformetrics_array_Kmeans.cluster_centers_\n    #print(tmp_cluster_centers)\n    tmp_centers_index = []\n    tmp_centers_metrics_name= []\n    #print(workload_list_afterFA[i].shape)\n    for j in range(num_M):\n        tmp_center = tmp_cluster_centers[j]\n        tmp_center_diff = tmp_center - workload_list_afterFA[i]\n        tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)\n        tmp_center_min_idx = np.argmin(tmp_center_distance)\n        tmp_centers_index.append(tmp_center_min_idx)\n        #print(tmp_center_min_idx)\n        #print(\'-----\')\n    #tmp_centers_metrics_name = []\n    tmp_n_centers = len(tmp_clust

In [1965]:
start_time = time.time()

#Step6 Get Prutiend workload (apply for all workloads)

knobs_latency_list = ['k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4']
latency = ['latency']
pruning_workload_list = []

for i in range(num_workloads_A):
    #tmp_pruning_list = knobs_latency_list + latency + tmp_centers_metrics_name
    #tmp_pruning_list = knobs_latency_list + latency + workload_list_centers_metrics_name[i]
    tmp_pruning_list = knobs_latency_list + workload_list_centers_metrics_name[i]
    tmp_pruning_workload_A = (workload_A_list[i])[tmp_pruning_list]
    pruning_workload_list.append(tmp_pruning_workload_A)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04419660568237305 seconds ---


In [1966]:
start_time = time.time()

#Step7 Building GPR (apply for all workloads)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler


tmp_kernel = 1.0 * RBF(length_scale=1.0)

workload_A_GPRmodels_lists = []
workload_A_scalar_lists = []
workload_A_normalized_m = []

for i in range(58):
    tmp_GPRmodels_lists = []
    tmp_scalar_lists = []
    tmp_normalized_m = []
    
    tmp_Pruning_workload_A_X = ((workload_A_list[i])[knobs_latency_list]).values
    tmp_Pruning_workload_A_y_o = ((workload_A_list[i])[latency]).values
    #print(tmp_Pruning_workload_A_y_o)
    
    tmp_x_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_X)
    tmp_y_o_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_y_o)
    
    tmp_scalar_lists.append(tmp_x_scaler)
    tmp_scalar_lists.append(tmp_y_o_scaler)
    
    tmp_Pruning_workload_A_X_normalized = tmp_x_scaler.transform(tmp_Pruning_workload_A_X)
    #print(tmp_x_scaler.data_min_)
    tmp_Pruning_workload_A_y_o_normalized = tmp_y_o_scaler.transform(tmp_Pruning_workload_A_y_o)
    
    tmp_normalized_m.append(tmp_Pruning_workload_A_y_o_normalized)
    #print(tmp_Pruning_workload_A_y_o_normalized)
    
    #tmp_kernel = DotProduct() + WhiteKernel()
    tmp_workload_A_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Pruning_workload_A_X_normalized, tmp_Pruning_workload_A_y_o_normalized)
    
    tmp_GPRmodels_lists.append(tmp_workload_A_gpr_o)
    
    tmp_num_centers = len(workload_list_centers_metrics_name[i])
    for j in range(tmp_num_centers):
        
        tmp_Pruning_workload_A_y_m = ((workload_A_list[i])[[workload_list_centers_metrics_name[i][j]]]).values
        #print(tmp_Pruning_workload_A_y_m)
        tmp_y_m_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_y_m)
        tmp_scalar_lists.append(tmp_y_m_scaler)
        tmp_Pruning_workload_A_y_m_normalized = tmp_y_m_scaler.transform(tmp_Pruning_workload_A_y_m)
        tmp_normalized_m.append(tmp_Pruning_workload_A_y_m_normalized)
        
        #tmp_kernel = DotProduct() + WhiteKernel()
        tmp_workload_A_gpr_m = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Pruning_workload_A_X_normalized, tmp_Pruning_workload_A_y_m_normalized)
        tmp_GPRmodels_lists.append(tmp_workload_A_gpr_m)
        
    workload_A_GPRmodels_lists.append(tmp_GPRmodels_lists)
    workload_A_scalar_lists.append(tmp_scalar_lists)
    workload_A_normalized_m.append(tmp_normalized_m)
print("--- %s seconds ---" % (time.time() - start_time))
#print(workload_A_normalized_m)
print(workload_A_scalar_lists[0])

--- 57.690635681152344 seconds ---
[MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1))]


In [1967]:
#*************************************************************************************************************************************************
#    1
#    We have workload_A_list contain 58 workloads
#    workload_A_list
#
#    2
#    We have workload_list_centers_metrics_name contain 58 lists, each list have the metrics kept after FA and K-means for each workload
#    workload_list_centers_metrics_name
#
#    3
#    We have pruning_workload_list contain 58 workloads after prunning
#    pruning_workload_list
#
#    4
#    We have workload_A_GPRmodels_lists contain 58 lists, each list have 1(latency) + k(number of metrics kept) GPR models
#    workload_A_GPRmodels_lists
#
#*************************************************************************************************************************************************

In [1968]:
workload_list_centers_metrics_name[0]

['latency',
 'worker_1.Disk_Block_Size.dm-0',
 'worker_2.Paging_and_Virtual_Memory.pgfree',
 'executor.jvm.pools.PS-Old-Gen.max.avg',
 'executor.jvm.pools.PS-Eden-Space.used.avg']

In [1969]:
#Phase2 Using the online workload B as the validation data

#In workloads B, each workload has 6 data points, we use 5 of them apply workload mapping, and use the left 1 as the target workload, predict
#latency and compare it with the actual value in workloads B.

In [1970]:
start_time = time.time()

#Step1 load workload B data set

workload_B=pd.read_csv('../../online_workload_B.csv') 

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.09374880790710449 seconds ---


In [1971]:
start_time = time.time()

#Step2 Split workload_B by creating different dataframes

workload_B_ids = workload_B["workload id"].unique()
workload_B_list = []
for num,workload_B_id in enumerate(workload_B_ids,start=1):
    tmp_workload = workload_B[workload_B['workload id'].isin([workload_B_id])]
    workload_B_list.append(tmp_workload)
    exec("workload_B_%s = tmp_workload"%num)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.07679533958435059 seconds ---


In [1972]:
start_time = time.time()

#Step3 Split each workload into 5 data points and 1 data point as target

num_workloads_B = len(workload_B_list)

workload_B_mapping_data = []
workload_B_target = []

for i in range(num_workloads_B):
    workload_B_mapping_data.append(workload_B_list[i][:-1])
    workload_B_target.append(workload_B_list[i][5:6])
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.03889584541320801 seconds ---


In [1973]:
start_time = time.time()

#Step4 workload mapping Test for all workload in workload B
all_distance_list = []
for i in range(100):
    tmp_test_workload_B_mapping = workload_B_mapping_data[i]
    tmp_test_workload_B_mapping_cfigs = ((tmp_test_workload_B_mapping)[knobs_latency_list]).values
    #print(tmp_test_workload_B_mapping_cfigs)
    tmp_distance_list = []
    
    for j in range(58):
        tmp_test_workload_A = workload_A_list[j]
        tmp_test_workload_A_metrics = workload_list_centers_metrics_name[j]
        tmp_test_workload_A_gpr_model = workload_A_GPRmodels_lists[j]
        tmp_test_prunning_workload_A = pruning_workload_list[j]
        tmp_test_workload_A_cigs_scalar = workload_A_scalar_lists[j][0]
        tmp_test_workload_A_normalized_m = workload_A_normalized_m[j]
        
        tmp_norm_workload_B_mapping_cfigs = tmp_test_workload_A_cigs_scalar.transform(tmp_test_workload_B_mapping_cfigs)
        #print(tmp_norm_workload_B_mapping_cfigs)
        #print(tmp_norm_workload_B_mapping_cfigs)
        tmp_test_workload_A_cfigs = ((tmp_test_workload_A)[knobs_latency_list]).values
        #print(tmp_test_workload_A_gpr_model)
        tmp_corr_metric_values = []
        
        for k in range(5):
            tmp_data_point = tmp_test_workload_B_mapping_cfigs[k]
            tmp_check = np.where((tmp_data_point==tmp_test_workload_A_cfigs[:,None]).all(-1))
            
            if len(tmp_check[0]) == 0:
                tmp_gpr_predic = []
                for v in range(num_M):
                    tmp_gpr_predic.append(tmp_test_workload_A_gpr_model[v+1].predict([tmp_norm_workload_B_mapping_cfigs[k]])[0])
                tmp_corr_metric_values.append(tmp_gpr_predic)
                #print(tmp_gpr_predic)
            
            else:
                tmp_gpr_predic = []
                tmp_row_idx = tmp_check[0][0]
                #print(tmp_row_idx)
                #print('xxxxxxxxxxxxxxxxxxx')
                tmp_test_prunning_workload_A_array = tmp_test_prunning_workload_A.values
                #print(tmp_test_workload_A_normalized_m)
                for n in range(num_M):
                    #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][13+n])
                    tmp_gpr_predic.append(tmp_test_workload_A_normalized_m[n+1][tmp_row_idx])
                    
                #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][13])
                #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][14])
                tmp_corr_metric_values.append(tmp_gpr_predic)
        
        tmp_test_workload_B_mapping_metrics_norm_value_list = []
        for m in range(num_M):
            tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[[tmp_test_workload_A_metrics[m]]].values
            tmp_test_workload_B_mapping_metrics_nomr_value = workload_A_scalar_lists[j][2+m].transform(tmp_test_workload_B_mapping_metrics_value)
            tmp_test_workload_B_mapping_metrics_norm_value_list.append(tmp_test_workload_B_mapping_metrics_nomr_value)
            
            
        #tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((2,5))
        #tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((2,5))
        #print(tmp_test_workload_B_mapping_metrics_norm_value_list)
        
        #print('---')
        #print(tmp_corr_metric_values)
        #print('xxxxxxxxxxxxxx')
        tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((num_M,5))       
        tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((5,num_M)).T
        #print(tmp_test_workload_B_mapping_metrics_norm_value_arry)
        #print('-----')
        #print(tmp_corr_metric_values_arry)
        
        #print('----------------')
        up_min_list = np.min(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
        down_min_list = np.min(tmp_corr_metric_values_arry,axis=0)
        tot_min_list = np.min(np.vstack((up_min_list,down_min_list)),axis=0)
        
        up_max_list = np.max(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
        down_max_list = np.max(tmp_corr_metric_values_arry,axis=0)
        tot_max_list = np.max(np.vstack((up_max_list,down_max_list)),axis=0)
        #print(up_min_list)
        #print(down_min_list)
        #print(tot_min_list)
        #test_min = tmp_test_workload_B_mapping_metrics_norm_value_arry - tot_min_list
        #print('xxxxxxxxxxx')
        #print(test_min)
        
        tmp_test_workload_B_mapping_metrics_norm_value_arry_norm = np.divide((tmp_test_workload_B_mapping_metrics_norm_value_arry - tot_min_list),(tot_max_list - tot_min_list))
        tmp_corr_metric_values_arry_norm = np.divide((tmp_corr_metric_values_arry - tot_min_list),(tot_max_list - tot_min_list))
        
        #print(tmp_test_workload_B_mapping_metrics_norm_value_arry_norm)
        
        #tmp_min = min(np.min(tmp_test_workload_B_mapping_metrics_norm_value_arry),np.min(tmp_corr_metric_values_arry))
        #tmp_max = max(np.max(tmp_test_workload_B_mapping_metrics_norm_value_arry),np.max(tmp_corr_metric_values_arry))
        #print(np.min(tmp_test_workload_B_mapping_metrics_norm_value_arry))
        #print(np.min(tmp_corr_metric_values_arry))
        #print(tmp_min)
        #print(tmp_max)
        
        #tmp_test_workload_B_mapping_metrics_norm_value_arry_norm = (tmp_test_workload_B_mapping_metrics_norm_value_arry-tmp_min)/(tmp_max-tmp_min)
        #tmp_corr_metric_values_arry_norm = (tmp_corr_metric_values_arry-tmp_min)/(tmp_max-tmp_min)
        
        #print(tmp_test_workload_B_mapping_metrics_norm_value_arry_norm)
        #print(tmp_corr_metric_values_arry_norm)
        
        #tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[tmp_test_workload_A_metrics].values
        #print(tmp_test_workload_B_mapping_metrics_value)
        #print('--------')
        tmp_distance = np.sum((tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm)*(tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm))
        #tmp_distance = np.sum((tmp_test_workload_B_mapping_metrics_norm_value_arry-tmp_corr_metric_values_arry)*(tmp_test_workload_B_mapping_metrics_norm_value_arry-tmp_corr_metric_values_arry))
        tmp_distance_list.append(tmp_distance)
     
    all_distance_list.append(tmp_distance_list)


    
all_min_idx_list = []
for i in range(100):
    tmp_idx = all_distance_list[i].index(min(all_distance_list[i]))
    all_min_idx_list.append(tmp_idx)

print("--- %s seconds ---" % (time.time() - start_time))


--- 39.99406695365906 seconds ---


In [1974]:
print(all_min_idx_list)

[10, 1, 2, 37, 35, 1, 38, 1, 8, 3, 42, 39, 12, 10, 41, 15, 12, 31, 52, 16, 38, 11, 55, 42, 8, 38, 39, 12, 14, 57, 53, 29, 32, 29, 40, 35, 8, 5, 48, 9, 10, 47, 56, 13, 26, 16, 15, 44, 37, 52, 44, 39, 22, 41, 44, 38, 36, 47, 8, 31, 42, 31, 2, 1, 24, 47, 44, 35, 8, 15, 12, 28, 38, 44, 44, 17, 54, 27, 20, 49, 50, 51, 52, 38, 39, 52, 38, 57, 41, 1, 2, 57, 49, 34, 26, 7, 38, 16, 15, 26]


In [1975]:
start_time = time.time()

#Step5 Concatenate trances from the observed target workload and the nearest neighbor workload for all workload B

predictGPRmodel_list = []
predictScalar_list = []
inputScalar_list = []

for i in range(100):
    tmp_similar_idx = all_min_idx_list[i]
    tmp_similar_workload = pruning_workload_list[tmp_similar_idx]
    tmp_similar_workload_configs = ((pruning_workload_list[tmp_similar_idx])[knobs_latency_list]).values
    tmp_similar_workload_metrics = ((pruning_workload_list[tmp_similar_idx])[workload_list_centers_metrics_name[tmp_similar_idx]]).values
    tmp_workload_B_mapping_metrics_with_similar = ((workload_B_mapping_data[i])[workload_list_centers_metrics_name[tmp_similar_idx]]).values
    #tmp_similar_workload_metrics = ((pruning_workload_list[tmp_similar_idx])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    #tmp_workload_B_mapping_metrics_with_similar = ((workload_B_mapping_data[i])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    
    tmp_Augmented_pruned_target_configs = np.array(tmp_similar_workload_configs)
    tmp_Augmented_pruned_target_metrics = np.array(tmp_similar_workload_metrics)
    
    (((pruning_workload_list[tmp_similar_idx])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]))
    
    #print(tmp_Augmented_pruned_target_configs)
    
    tmp_workload_B_configs = workload_B_mapping_data[i][knobs_latency_list].values
    #print(tmp_workload_B_configs)
    
    for j in range(5):
        tmp_test_workload_B_mapping_cfigs = tmp_workload_B_configs[j]
        tmp_test_workload_B_mapping_metrics_with_similar = tmp_workload_B_mapping_metrics_with_similar[j]
        tmp_check = np.where((tmp_test_workload_B_mapping_cfigs==tmp_similar_workload_configs[:,None]).all(-1))
        
        if len(tmp_check[0]) == 0:
            #print('z')
            #print(tmp_test_workload_B_mapping_cfigs)
            #print(tmp_test_workload_B_mapping_metrics_with_similar)
            #tmp_Augmented_pruned_target_configs = np.insert(tmp_Augmented_pruned_target_configs,0,tmp_test_workload_B_mapping_cfigs,axis=0)
            
            #tmp_Augmented_pruned_target_metrics = np.insert(tmp_Augmented_pruned_target_metrics,0,tmp_test_workload_B_mapping_metrics_with_similar,axis=0)
            
            tmp_Augmented_pruned_target_configs = np.append(tmp_Augmented_pruned_target_configs,[tmp_test_workload_B_mapping_cfigs],axis=0)
            tmp_Augmented_pruned_target_metrics = np.append(tmp_Augmented_pruned_target_metrics,[tmp_test_workload_B_mapping_metrics_with_similar],axis=0)
            
            #print(tmp_Augmented_pruned_target_configs)
            #print(tmp_Augmented_pruned_target_metrics)
        else:
        
            tmp_row_idx = tmp_check[0][0]
            #print(tmp_row_idx)
            #print('xxxxxxxxxxxxxxxxxxx')
            tmp_Augmented_pruned_target_metrics[tmp_row_idx] = tmp_test_workload_B_mapping_metrics_with_similar
    
    tmp_metrics_o_size = tmp_Augmented_pruned_target_metrics.T[0].shape[0]
    tmp_nor_tmp_Augmented_pruned_target_metrics_o = tmp_Augmented_pruned_target_metrics.T[0].reshape((tmp_metrics_o_size,1))
    #print(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    tmp_o_scalar = MinMaxScaler().fit(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    predictScalar_list.append(tmp_o_scalar)
    
    tmp_Augmented_pruned_target_metrics_nomr = tmp_o_scalar.transform(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    tmp_X_scalar = MinMaxScaler().fit(tmp_Augmented_pruned_target_configs)
    inputScalar_list.append(tmp_X_scalar)
    
    tmp_Augmented_pruned_target_configs_norm = tmp_X_scalar.transform(tmp_Augmented_pruned_target_configs)
    #print(tmp_Augmented_pruned_target_metrics_nomr)
    
    #print(tmp_Augmented_pruned_target_configs)
    #print('------')
    #print(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    test_workload_B_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Augmented_pruned_target_configs_norm, tmp_Augmented_pruned_target_metrics_nomr)
    predictGPRmodel_list.append(test_workload_B_gpr_o)
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 18.504575967788696 seconds ---


In [1979]:
start_time = time.time()

#Step6 predict for all workload B
right_count = 0
for i in range(100):
    tmp_target_config = workload_B_target[i][knobs_latency_list].values
    #print(tmp_target_config)
    tmp_target_config_norm = inputScalar_list[i].transform(tmp_target_config)
    #print(tmp_target_config_norm)
    tmp_max = (predictScalar_list[i].data_max_[0])
    tmp_min = (predictScalar_list[i].data_min_[0])
    tmp_predict = predictGPRmodel_list[i].predict(tmp_target_config_norm)
    tmp_predict_re = tmp_predict * (tmp_max - tmp_min) + tmp_min
    tmp_diff = np.abs(tmp_predict_re - workload_B_target[i][latency].values)
    print('Predict: ',tmp_predict_re)
    print('Ground Truth: ',workload_B_target[i][latency].values)
    print('diff: ',tmp_diff)
    if (tmp_diff/workload_B_target[i][latency].values) < 0.15:
        right_count += 1
    print('------------')
    
print('Right Predict: ', right_count)
    
print("--- %s seconds ---" % (time.time() - start_time))


Predict:  [[1.54625]]
Ground Truth:  [[7.923]]
diff:  [[6.37675]]
------------
Predict:  [[261.46899986]]
Ground Truth:  [[260.62625]]
diff:  [[0.84274986]]
------------
Predict:  [[111.98174991]]
Ground Truth:  [[109.0855]]
diff:  [[2.89624991]]
------------
Predict:  [[22.54225]]
Ground Truth:  [[61.997]]
diff:  [[39.45475]]
------------
Predict:  [[149.14574985]]
Ground Truth:  [[89.78475]]
diff:  [[59.36099985]]
------------
Predict:  [[177.92524987]]
Ground Truth:  [[24.4785]]
diff:  [[153.44674987]]
------------
Predict:  [[4.85225]]
Ground Truth:  [[4.507125]]
diff:  [[0.345125]]
------------
Predict:  [[40.08875]]
Ground Truth:  [[107.993]]
diff:  [[67.90425]]
------------
Predict:  [[42.71249996]]
Ground Truth:  [[42.31425]]
diff:  [[0.39824996]]
------------
Predict:  [[56.61299999]]
Ground Truth:  [[52.674]]
diff:  [[3.93899999]]
------------
Predict:  [[2.168]]
Ground Truth:  [[3.084]]
diff:  [[0.916]]
------------
Predict:  [[9.4355]]
Ground Truth:  [[9.70575]]
diff:  [[0.